# Imports

# Sentiment Analysis

### NLTK Approach

In [11]:
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()
scores = analyzer.polarity_scores(text)
positive_percent = scores['pos'] * 100
negative_percent = scores['neg'] * 100

print("Sentiment Analysis Results:")
print(f"Positive Sentiment: {positive_percent:.2f}%")
print(f"Negative Sentiment: {negative_percent:.2f}%")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\haris\AppData\Roaming\nltk_data...


Sentiment Analysis Results:
Positive Sentiment: 65.40%
Negative Sentiment: 0.00%


### Bert base model approach

In [13]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

c:\Users\haris\miniconda3\envs\forPyTorch\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\haris\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [15]:
inputs = tokenizer(text=text, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
temp = model.config.id2label[predicted_class_id]

In [16]:
temp

'POSITIVE'

# Summary and Topic

In [10]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import speech_recognition as sr
import pydub
from googletrans import Translator
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
from google.cloud import speech
import subprocess
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
from flask import Flask, request, jsonify
from flask_cors import CORS
from pydub import AudioSegment 
import speech_recognition as sr 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from googletrans import Translator  
from transformers import pipeline


app = Flask(__name__)
CORS(app, origins="http://localhost:3000", supports_credentials=True)


@app.route('/api/convert-mp3-to-text', methods=['POST'])
def convert_mp3_to_text():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file: 
        mp3_filename = os.path.join('uploads', file.filename)
        file.save(mp3_filename)
 
        wav_filename = mp3_filename.replace('.mp3', '.wav')
        AudioSegment.from_mp3(mp3_filename).export(wav_filename, format="wav")
 
        recognizer = sr.Recognizer()
        with sr.AudioFile(wav_filename) as source:
            audio = recognizer.record(source)

        try:
            text = recognizer.recognize_google(audio)
            return jsonify({'text': text})
        except sr.UnknownValueError:
            return jsonify({'error': 'Could not understand audio'}), 400
        except sr.RequestError as e:
            return jsonify({'error': f'Speech Recognition error: {e}'}), 500
        

@app.route('/api/translate_toar/<string:text>', methods=['GET'])
def translate_to_ar(text):    
    translator = Translator() 
    arabic_translation = translator.translate(text, src='en', dest='ar').text
    return jsonify({'translated_txt': arabic_translation})


@app.route('/api/translate_totr/<string:text>', methods=['GET'])
def translate_to_tr(text):  
    translator = Translator()    
    turkish_translation = translator.translate(text, src='en', dest='tr').text
    return jsonify({'translated_txt': turkish_translation})

@app.route('/api/findtopic/<string:text>', methods=['GET'])
def topic_finder(text):
    pipe = pipeline("summarization", model="google/pegasus-xsum")
    topic = pipe(text, max_length = 5)
    topic = topic[0]['summary_text']
    return jsonify({'topic': topic})

@app.route('/api/findSummary/<string:text>', methods=['GET'])
def summary_find(text):
    pipe = pipeline("summarization", model="google/pegasus-xsum")
    output = pipe(text, max_length = 30)
    final_summary = output[0]['summary_text']

    translator = Translator() 
    arabic_summary = translator.translate(final_summary, src='en', dest='ar').text
    turkish_summary = translator.translate(final_summary, src='en', dest='tr').text

    return jsonify({'summary_en': final_summary, 'summary_ar': arabic_summary, 'summary_tr': turkish_summary})

if __name__ == '__main__': 
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [18/Oct/2023 19:43:46] "POST /api/convert-mp3-to-text HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2023 19:43:48] "GET /api/findtopic/ HTTP/1.1" 404 -
127.0.0.1 - - [18/Oct/2023 19:44:36] "GET /api/translate_to/American%20accent%20in%2010%20seconds%20is%20it%20is%20saying%20is%20he%20nice%20say%20easy%20nice%20easy%20nice%20easy%20nice%20nice HTTP/1.1" 404 -
127.0.0.1 - - [18/Oct/2023 19:44:41] "GET /api/translate_totr/American%20accent%20in%2010%20seconds%20is%20it%20is%20saying%20is%20he%20nice%20say%20easy%20nice%20easy%20nice%20easy%20nice%20nice HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2023 19:46:51] "POST /api/convert-mp3-to-text HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2023 19:46:56] "GET /api/findtopic/ HTTP/1.1" 404 -
127.0.0.1 - - [18/Oct/2023 19:47:44] "POST /api/convert-mp3-to-text HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2023 19:47:49] "GET /api/findtopic/ HTTP/1.1" 404 -
127.0.0.1 - - [18/Oct/2023 19:50:08] "POST /api/convert-mp3-to-text HTTP/1.1" 500 -
127.0.0.1 - - [18/Oc